# Proofread in eCREST

The files generated by this script will also be able to be opened in CREST original (though some information may be lost if using original CREST.py or .exe).

## Setup

Do the following two setup steps regardless of how you will be using this script. 

### 1. Imports

Run the following code cell to import the necessary packages and modules. 

In [ ]:
############################################################################################################################ 
# Get the latest CREST files for each ID within the target folder (dirname)

from pathlib import Path
import json
from sqlite3 import connect as sqlite3_connect
from sqlite3 import DatabaseError
from igraph import Graph as ig_Graph
from igraph import plot as ig_plot
from scipy.spatial.distance import cdist
from random import choice as random_choice
from itertools import combinations
from numpy import array, unravel_index, argmin, mean,unique,nan
import pandas as pd
from copy import deepcopy
from datetime import datetime
from time import time
import neuroglancer
from webbrowser import open as wb_open
from webbrowser import open_new as wb_open_new
import neuroglancer

# from eCREST_cli_beta import ecrest, import_settings
from eCREST_cli import ecrest, import_settings

The 'ecrest' class has been imported from eCREST_cli.py

An instance of this object will be able to:
- open an neuroglancer viewer for proofrieading (see "Proofread using CREST")
    - add-remove segments (using graph feature for efficiency)
    - format itself and save itself as a CREST-style .json
- convert from neuroglancer json (see "Convert From Neuroglancer to eCREST")
    - format itself and save itself as a CREST-style .json
    


# USING THE CREST_JSON class

## Settings definitions

Whether you are converting from neuroglancer or creating a new reconstruction, the settings_dict parameters is needed to create CREST json files with correct formatting. 
- 'save_dir' : the directory where JSON files are saved 
- 'cred' and 'db_path' : specify the path to the agglomeration database file on your local computer. 

In [ ]:
settings_dict = {
    'save_dir' : '/Users/kperks/Documents/eCREST-local-files/in-progress',
    'db_path' : '/Users/kperks/Documents/eCREST-local-files/Mariela_bigquery_exports_agglo_v230111c_16_crest_proofreading_database.db',
    'max_num_base_added' : 1000,
    'cell_structures' : ['unknown','axon', 'basal dendrite', 'apical dendrite', 'dendrite', 'multiple'],
    'annotation_points' : ['exit volume', 'natural end', 'uncertain', 'pre-synaptic', 'post-synaptic']
}

### Import settings

If you save a copy of settings_dict.json (found in the "under construction" directory of eCREST repo) locally somewhere outside the repo (like in your save_dir), then you can use the following code cell to import. This avoids needing to re-type the save_dir and db_path each time you "git pull" updates from the repo to this notebook.

In [ ]:
path_to_settings_json = '/Users/kperks/Documents/ell-connectome/eCREST-local-files/settings_dict.json'
settings_dict = import_settings(path_to_settings_json)

## Proofread using eCREST



### 1. Create a crest_json object that launches a proofreading instance of neuroglancer


Initialize with either:
- (segment_id, segment_list): the main_base_id from the neuroglancer file you are converting and a list of base_segments.
- (segment_id): a "main_base_id"
- (filepath): an existing CREST json file

#### NEW reconstruction from segment ID

If you wanted to start reconstructing a new cell from a main base segment, 
you would use the following code block to launch

In [ ]:
segment_id = 41501459#  # a manageable afferent
crest = ecrest(settings_dict,segment_id = segment_id, launch_viewer=True)
# viewer_object = crest.neuroglancer_viewer()
# crest.load_to_viewer()

'''
If you want to change keybindings for functions:
'''
with crest.viewer.config_state.txn() as s:
    s.input_event_bindings.data_view["alt+mousedown0"]="add-or-remove-seg"
    s.input_event_bindings.data_view["alt+mousedown2"]="mark-branch-in-colour"
    print(s.input_event_bindings.data_view)

In [ ]:
# crest.max_num_base_added=2000
crest.save_cell_graph()

#### EDIT reconstruction from file

If you wanted to edit a reconstruction from an existing file, 
you would use the following code block to launch

In [ ]:
dirpath = Path(settings_dict['save_dir'])

nodefiles = dict()
for child in sorted(dirpath.iterdir()):
    if (child.name[0]!='.') & (child.is_file()):
        nodefiles[child.name.split('_')[2]] = child

In [ ]:
['', '', '', '']

In [ ]:
crest = ecrest(settings_dict,filepath= nodefiles['214613070'], launch_viewer=True)

In [ ]:
todo = Path(settings_dict['save_dir']) /'todo_presynaptic/mg2_214581797'
len([child for child in todo.iterdir()])

In [ ]:
json_path = Path(settings_dict['save_dir']) /'todo_presynaptic/mg2_214581797' #/ 'kp/482680782_grc'##/ 'todo_postsynaptic_sg' #
filename = 'cell_graph_210034272__2023-07-15 09.50.19.json'#'cell_graph_306242528__2023-06-26 09.26.24.json'

crest = ecrest(settings_dict,filepath= json_path / filename, launch_viewer=True)


'''
If you want to change keybindings for functions:
'''
with crest.viewer.config_state.txn() as s:
    s.input_event_bindings.data_view["alt+mousedown0"]="add-or-remove-seg"
    s.input_event_bindings.data_view["alt+mousedown2"]="mark-branch-in-colour"
    print(s.input_event_bindings.data_view)

# # crest.cell_data['removed_base_segs']=set()
crest.max_num_base_added=1500

# crest.cell_data['removed_base_segs']=set()

## check for duplicates (single cell)

In [ ]:
base_segments_net = crest.get_base_segments_dict(Path(settings_dict['save_dir']))
# base_segments_kp = crest.get_base_segments_dict(Path(settings_dict['save_dir']) / 'kp/482680782_grc')
base_segments_mg1 = crest.get_base_segments_dict(Path(settings_dict['save_dir']) / 'todo_postsynaptic_sg')
base_segments_aff = crest.get_base_segments_dict(Path(settings_dict['save_dir']) / 'todo_afferent')
base_segments_mg2_p = crest.get_base_segments_dict(Path(settings_dict['save_dir']) / 'todo_presynaptic/mg2_214581797')
base_segments_mg1_p = crest.get_base_segments_dict(Path(settings_dict['save_dir']) / 'todo_presynaptic/mg1_299496636')
base_segments_p = crest.get_base_segments_dict(Path(settings_dict['save_dir']) / 'todo_presynaptic')
base_segments_postmg = crest.get_base_segments_dict(Path(settings_dict['save_dir']) / 'todo_postsynaptic_mg')
base_segments_postmg_check = crest.get_base_segments_dict(Path(settings_dict['save_dir']) / 'todo_postsynaptic_mg/check-duplicates')

In [ ]:
print('from the main folder:')
df = crest.check_duplicates(base_segments_net)
display(df)
# print('from the kp folder:')
# df = crest.check_duplicates(base_segments_kp)
# display(df)
print('from the sg post folder:')
df = crest.check_duplicates(base_segments_mg1)
display(df)
print('from the aff folder:')
df = crest.check_duplicates(base_segments_aff)
display(df)
print('from the mg2 ex folder:')
df = crest.check_duplicates(base_segments_mg2_p)
display(df)
print('from the mg1 ex folder:')
df = crest.check_duplicates(base_segments_mg1_p)
display(df)
print('from the presyn folder:')
df = crest.check_duplicates(base_segments_p)
display(df)

# print('from the mg post folder:')
# df = crest.check_duplicates(base_segments_postmg)
# display(df)
# print('from the mg post check duplicates folder:')
# df = crest.check_duplicates(base_segments_postmg_check)
# display(df)

In [ ]:
# Assign the cell type then run the code cell
cell_type = 'vmi'

## Do not edit
method = 'manual' # define which method you are using (manual or auto)
crest.define_ctype(cell_type,method)

### 2. SAVE YOUR WORK BEFORE CLOSING NEUROGLANCER! 

In [ ]:
crest.save_cell_graph(directory_path = Path(settings_dict['save_dir']))# / 'todo_postsynaptic_sg/check-duplicates')#/'volume-subsample-all/in-progress')# / 'todo_presynaptic/sg1')

If you want to re-write the file you opened instead of saving with a new timestamp in the filename, run the following code cell instead of the previous one.

In [ ]:
filepath = json_path / filename
crest.save_cell_graph(directory_path = filepath.parent)#, file_name=filepath.name); 

#### Use the following to open a new cell in the same neuroglancer tab as is already opened

**DOES NOT WORK YET**

In [ ]:
# json_path = Path(settings_dict['save_dir']) / 'todo_post-synaptic'
# filename = 'cell_graph_302637877__2023-04-09 19.21.28.json'

# crest = ecrest(settings_dict,filepath= json_path / filename)
# crest.neuroglancer_viewer(viewer_object)
# crest.load_to_viewer()


### 3. CELL TYPING

If part of your job as a reconstructor is to identify cell types, then you can use the following blocks of code.  
First, check if it is already defined (and what the cell type was defined as).  


After you are finished defining the cell type:  
**DONT FORGET TO SAVE YOUR WORK!**. 
(step 2)

In [ ]:
# Assign which method you are using (manual or auto)
method = 'manual'

## Do not edit
crest.get_ctype(method)

If not defined (or defined incorrectly), then define it.
> OPTIONS: mg1, mg2, mgx, lg, lf, lx, mli, gc, gran, sg

In [ ]:
# Assign the cell type and which method you are using (manual or auto)
cell_type = 'lf'
method = 'manual'

## Do not edit
crest.define_ctype(cell_type,method)

## Check for DUPLICATES - single cell

Specify a folder of cells that you want to check for duplicates with the cell you are reconstructing.

The following code cell uses the function ```get_base_segments_dict``` in the crest instance to create a dictionary of all base segments for each cell within a specified directory. 

In [ ]:
dirpath = Path(settings_dict['save_dir']) #/ 'todo_post-synaptic'

base_segments = crest.get_base_segments_dict(dirpath)


Load a cell that needs checking

In [ ]:
json_path = Path(settings_dict['save_dir']) / 'todo_post-synaptic' / 'check-duplicates'
filename = 'cell_graph_50844566__2023-04-21 14.47.44.json'

crest = ecrest(settings_dict,filepath= json_path / filename, launch_viewer=False)

And then uses the function ```get_duplicates``` in the crest instance to check if it overlaps with any of the cells in that directory.

In [ ]:
df = crest.check_duplicates(base_segments)

display(df)

## Convert From Neuroglancer to eCREST

Run the following code cell to convert neuroglancer json files to eCREST json files. 

Uses "conversion_specs.json" to batch process conversion.

Conversion using "conversion_specs.json" expects:
- a folder of neuroglancer json files (with filenames standardized like in Google Drive)
- "dirname" is the folder containing neuroglancer json files to be converted
- that the "conversion_specs.json" is in the ```settings_dict['save_dir']``` key

### Batch

In [ ]:
conversion_specs_filename = "conversion_specs.json"

with open(Path(settings_dict['save_dir']) / conversion_specs_filename) as f:
    conversion_specs = json.load(f)

p = Path(conversion_specs['dirname'])

for cell_id, info in conversion_specs['cell_info'].items():
   
    f = info['filename']
    neuroglancer_layer_name = info['neuroglancer_layer_name']
    crest_layer_name = info['crest_layer_name']
  
    ## Get main_base_seg_ID from filename or from list of segment IDs
    main_base_id = f.split('_')[1] # gets the base segment ID from the name
    
    try:
        assert cell_id == main_base_id, f'cell id and filename do not match in conversion json; moving on to next cell without completing this one'
    except AssertionError as msg:
        print(msg)
        #add error message to json
        with open(settings_dict['save_dir'] / conversion_specs_filename, "r") as f:
            loaded = json.load(f)
        loaded['cell_info'][cell_id]['errors'].append(str(msg))
        with open(settings_dict['save_dir'] / conversion_specs_filename, "w") as f:
            json.dump(loaded, f, indent=4)
        continue
    
    ## Load the neuroglancer json
    print(f'you have selected cell {cell_id} to convert')
    
    with open(p / f, 'r') as myfile: # 'p' is the dirpath and 'f' is the filename from the created 'd' dictionary
        neuroglancer_data = json.load(myfile)

    print(f'Obtaining base_seg IDs from segmentation layer of neuroglancer json.')

    ## Obtain the list of base_segments from the neuroglancer json
    segmentation_layer = next((item for item in neuroglancer_data['layers'] if item["source"] == 'brainmaps://10393113184:ell:roi450um_seg32fb16fb_220930'), None)
    try:
        # add annotation layer
        
        base_segment_list_ng = segmentation_layer['segments']
    except TypeError as msg:
        print(msg, f': segmentation layer source is different; moving on to next cell without completing this one')
        #add error message to json
        with open(settings_dict['save_dir'] / conversion_specs_filename, "r") as f:
            loaded = json.load(f)
        loaded['cell_info'][cell_id]['errors'].append(str(msg) + f': segmentation layer source is different; moving on to next cell without completing this one')
        with open(settings_dict['save_dir'] / conversion_specs_filename, "w") as f:
            json.dump(loaded, f, indent=4)
        continue

    
    

    print(f'creating a crest_json object with no viewer for this cell')
    ## Create CREST instance with no viewer, segment_list, and segment_id
    crest = ecrest(settings_dict, segment_id = main_base_id, segment_list = base_segment_list_ng, launch_viewer=False)

    print(f'importing annotation layers from neuroglancer')
    ## Get annotations from neuroglancer -- iterate through one layer at a time to check for errors in layer names
    for nl_, cl_ in zip(neuroglancer_layer_name, crest_layer_name):

        # get the 'layers' dictionary that has that name
        neuroglancer_layer = next((item for item in neuroglancer_data['layers'] if item["name"] == nl_), None)

        if neuroglancer_layer != None:
            if cl_ in crest.point_types:
                # add annotation layer
                crest.import_annotations(neuroglancer_data, [nl_], [cl_])
                print(f"Imported - {nl_} - layer from neuroglancer annotations tabs for cell {crest.cell_data['metadata']['main_seg']['base']} as - {cl_} -.")
            else: 
                msg = f"CREST layer name - {cl_} - incorrect for cell {crest.cell_data['metadata']['main_seg']['base']} in conversion_json"
                print(msg)
                #add error message to json
                with open(crest.save_dir / conversion_specs_filename, "r") as f:
                    loaded = json.load(f)
                loaded['cell_info'][cell_id]['errors'].append(str(msg))
                with open(crest.save_dir / conversion_specs_filename, "w") as f:
                    json.dump(loaded, f, indent=4)
        else:
            msg = f"no layer by the name - {nl_} - in neuroglancer json for cell {crest.cell_data['metadata']['main_seg']['base']}"
            print(msg)
            #add error message to json
            with open(crest.save_dir / conversion_specs_filename, "r") as f:
                loaded = json.load(f)
            loaded['cell_info'][cell_id]['errors'].append(str(msg))
            with open(crest.save_dir / conversion_specs_filename, "w") as f:
                json.dump(loaded, f, indent=4)


    ## Save the cell_data as json
    print(f'saving cell {cell_id} with completed graph and annotations layers imported')
    crest.save_cell_graph() # If do not give file_path, then it will auto-generate one like CREST produces

### Single file

Just make the "conversion_specs" file have one cell in it. The "batch" loop will still run on one cell. 

### Just annotations layer

If starting from scratch on a reconstruction is faster than converting the base_segs into a graph... but you want the annotations preserved.

#### From another crest file

In [ ]:
json_path = Path(settings_dict['save_dir']) #/ 'todo_post-synaptic'
filename = 'cell_graph_299497999__2023-06-29 11.15.50.json'

crest_ = ecrest(settings_dict,filepath= json_path / filename, launch_viewer=True)

In [ ]:
for layer_name in crest.cell_data['end_points'].keys():
    crest.cell_data['end_points'][layer_name] = crest_.cell_data['end_points'][layer_name]

    crest.load_annotation_layer_points()

#### from neuroglancer file

In [ ]:
json_path = Path(settings_dict['save_dir']) #/ 'todo_post-synaptic'
filename = 'cell_graph_482680782__2023-07-20 17.46.00.json'

crest = ecrest(settings_dict,filepath= json_path / filename, launch_viewer=True)

In [ ]:
neuroglancer_layer_name = ['post-synaptic']#'pre-synaptic']#,
crest_layer_name = ['pre-synaptic']#,'post-synaptic']
neuroglancer_path = '/Users/kperks/Documents/gdrive/.shortcut-targets-by-id/16q1BuOMfD2ta0Cwq8CjMlRe4rDvbuWC5/ELL_connectome/CREST_reconstructions/mg-network/Nate_neuroglancer_synapses/finished'
neuroglancer_path = Path(neuroglancer_path) / '482680782_grc_nbs.json'

with open(Path(neuroglancer_path), 'r') as myfile: # 'p' is the dirpath and 'f' is the filename from the created 'd' dictionary
    neuroglancer_data = json.load(myfile)
# for nl_, cl_ in zip(neuroglancer_layer_name, crest_layer_name):

for nl_, cl_ in zip(neuroglancer_layer_name, crest_layer_name):
    # get the 'layers' dictionary that has that name
    neuroglancer_layer = next((item for item in neuroglancer_data['layers'] if item["name"] == nl_), None)

    if neuroglancer_layer != None:
        if cl_ in crest.point_types:
            # add annotation layer
            crest.import_annotations(neuroglancer_data, [nl_], [cl_])
            print(f"Imported - {nl_} - layer from neuroglancer annotations tabs for cell {crest.cell_data['metadata']['main_seg']['base']} as - {cl_} -.")
        else: 
            msg = f"CREST layer name - {cl_} - incorrect for cell {crest.cell_data['metadata']['main_seg']['base']} in conversion_json"
            print(msg)

    else:
        msg = f"no layer by the name - {nl_} - in neuroglancer json for cell {crest.cell_data['metadata']['main_seg']['base']}"
        print(msg)

crest.load_annotation_layer_points()

In [ ]:
segmentation_layer = next((item for item in neuroglancer_data['layers'] if item["source"] == 'brainmaps://10393113184:ell:roi450um_seg32fb16fb_220930'), None)
base_segment_list_ng = segmentation_layer['segments']

base_ids_added = set()

anchor_seg = crest.cell_data['metadata']['main_seg']['base']

segs_to_add = set(base_segment_list_ng).difference(set([a for b in crest.cell_data['base_segments'].values() for a in b]))


segs_to_add = [s for s in segs_to_add if '!' not in s]

print(len(segs_to_add))

In [ ]:
len(crest.cell_data['end_points']['post-synaptic'])

In [ ]:
## Save the cell_data as json
print(f'saving cell {neuroglancer_path} with annotations layers imported')
crest.save_cell_graph() # If do not give file_path, then it will auto-generate one like CREST produces

In [ ]:
base_ids_added = set()

for base_seg in segs_to_add: #overlap_segs["dup"].difference(overlap_segs["main"]): # dup diff main adds segments in dup that were not in main
    
    if (base_ids_added&set(base_seg)==set()) & (base_seg != crest.cell_data['metadata']['main_seg']['base']): 
        
        displayed_segs = crest.assert_segs_in_sync(return_segs=True)
        if base_seg in displayed_segs:
            # print(f'{base_seg} already in cell, continueing')
            continue

        # print(i,base_seg)
        agglo_seg = crest.get_agglo_seg_of_base_seg(base_seg)

        constituent_base_ids = crest.get_base_segs_of_agglo_seg(agglo_seg)        
        current_segs = crest.assert_segs_in_sync(return_segs=True)

        num_base_segs_this_agglo_seg = len(constituent_base_ids)
        constituent_base_ids = [x for x in constituent_base_ids if x not in current_segs]
        constituent_base_ids = [x for x in constituent_base_ids if x not in crest.cell_data['removed_base_segs']]
        num_base_segs_not_already_included = len(constituent_base_ids)
        
        if len(constituent_base_ids) > crest.max_num_base_added:
            base_ids = [base_seg]
            # crest.large_agglo_segs.add(agglo_seg)
            print(f'{len(constituent_base_ids)} other base segments in the agglo segment; max number can add is {crest.max_num_base_added}')
            # print(f'{base_seg} part of an agglo seg {agglo_seg} that is too large to add, so just adding the one segment')
        else:
            base_ids = constituent_base_ids

        if num_base_segs_this_agglo_seg > num_base_segs_not_already_included:

            if not base_seg in base_ids:
                base_ids.append(base_seg)
        print(base_ids)
        crest.update_base_locations(base_ids)
        crest.pr_graph.add_vertices(base_ids)

        if len(base_ids) > 1:
            edges = crest.get_edges_from_agglo_seg(agglo_seg)
            edges = [x for x in edges if (x[0] in base_ids and x[1] in base_ids)]
            crest.pr_graph.add_edges(edges)

        join_msg = crest.add_closest_edge_to_graph(base_ids, base_seg) 
        

        # Update lists of base segments and displayed segs:
        crest.cell_data['base_segments']['unknown'].update(set(base_ids))

        with crest.viewer.txn(overwrite=True) as s:

            for bs in base_ids:
                s.layers['base_segs'].segment_colors[int(bs)] = '#d2b48c'
                s.layers['base_segs'].segments.add(int(bs))
                
        base_ids_added.update(base_ids)


        crest.update_displayed_segs() 
        crest.assert_segs_in_sync()

#### dictionary of just one annotation layer 

(the json file would be a list of dicts.... each dict is an annotation point)

In [ ]:
annotations_path = json_path / 'tmp' /'annotations.json'

with open(annotations_path, 'r') as myfile: # 'p' is the dirpath and 'f' is the filename from the created 'd' dictionary
    annotate_data = json.load(myfile)
# for nl_, cl_ in zip(neuroglancer_layer_name, crest_layer_name):

# annotate_data
annotation_list = []
for v in annotate_data:


    # for v in neuroglancer_layer['annotations']:
    corrected_location = crest.get_corrected_xyz(v['point'], 'seg')

    if 'segments' not in v.keys():
        annotation_list.extend([corrected_location])
    if 'segments' in v.keys():
        annotation_list.extend([corrected_location + v['segments'][0]])

# self.cell_data['end_points'][c].extend(annotation_list)

crest.cell_data['end_points']['post-synaptic'] = annotation_list

crest.load_annotation_layer_points()

In [ ]:
## Save the cell_data as json
print(f'saving cell {neuroglancer_path} with annotations layers imported')
crest.save_cell_graph() # If do not give file_path, then it will auto-generate one like CREST produces

### segments from an NG json into an existing CREST

In [ ]:
json_path = Path(settings_dict['save_dir']) #/ 'todo_post-synaptic'
filename = 'cell_graph_476801247__2023-06-04 20.32.28.json'

crest = ecrest(settings_dict,filepath= json_path / filename, launch_viewer=True)

In [ ]:
neuroglancer_path = '/Users/kperks/Documents/gdrive/.shortcut-targets-by-id/16q1BuOMfD2ta0Cwq8CjMlRe4rDvbuWC5/ELL_connectome/CREST_reconstructions/mg-network/Nate_neuroglancer_synapses/finished'
neuroglancer_path = Path(neuroglancer_path) / '304356725_nbs.json'

with open(Path(neuroglancer_path), 'r') as myfile: # 'p' is the dirpath and 'f' is the filename from the created 'd' dictionary
    neuroglancer_data = json.load(myfile)

In [ ]:
segmentation_layer = next((item for item in neuroglancer_data['layers'] if item["source"] == 'brainmaps://10393113184:ell:roi450um_seg32fb16fb_220930'), None)
base_segment_list_ng = segmentation_layer['segments']



In [ ]:
base_ids_added = set()
anchor_cell = crest
anchor_seg = anchor_cell.cell_data['metadata']['main_seg']['base']

segs_to_add = set(base_segment_list_ng).difference(set([a for b in anchor_cell.cell_data['base_segments'].values() for a in b]))
print(len(segs_to_add))

In [ ]:
segs_to_add = set([x for x in list(segs_to_add) if "!" not in x ])

In [ ]:
'''this might be a version that does not work so well, see visualize overlapping segments section for one that does?'''
# for base_seg in segs_to_add:
#     # if this segment has not already been added and it is not the anchor seg_ (should not be if not already part of cell)
#     if (base_ids_added&set(base_seg)==set()) & (base_seg != anchor_seg): 
        
#         displayed_segs = anchor_cell.assert_segs_in_sync(return_segs=True)
#         if base_seg in displayed_segs:
#             # print(f'{base_seg} already in cell, continueing')
#             continue

#         # print(i,base_seg)
#         agglo_seg = anchor_cell.get_agglo_seg_of_base_seg(base_seg)

#         constituent_base_ids = anchor_cell.get_base_segs_of_agglo_seg(agglo_seg)
#         print(f'{len(constituent_base_ids)} other base segments in the agglo segment; max number can add is {crest.max_num_base_added}')


#         if len(constituent_base_ids) > anchor_cell.max_num_base_added:
#             base_ids = [base_seg]
#             # anchor_cell.large_agglo_segs.add(agglo_seg)
#             # print(f'{base_seg} part of an agglo seg {agglo_seg} that is too large to add, so just adding the one segment')
#         else:
#             base_ids = constituent_base_ids
        
#         current_segs = anchor_cell.assert_segs_in_sync(return_segs=True)

#         num_base_segs_this_agglo_seg = len(base_ids)
#         base_ids = [x for x in base_ids if x not in current_segs]
#         num_base_segs_not_already_included = len(base_ids)
        
#         # if there were segments from this agglo seg that were not in current graph, make sure you don't actually want them excluded
#         if num_base_segs_this_agglo_seg > num_base_segs_not_already_included:

#             base_ids = [x for x in base_ids if x not in anchor_cell.cell_data['removed_base_segs']]

#             if not base_seg in base_ids:
#                 base_ids.append(base_seg)
        
#         anchor_cell.update_base_locations(base_ids)
#         anchor_cell.pr_graph.add_vertices(base_ids)

#         if len(base_ids) > 1:
#             edges = anchor_cell.get_edges_from_agglo_seg(agglo_seg)
#             edges = [x for x in edges if (x[0] in base_ids and x[1] in base_ids)]
#             anchor_cell.pr_graph.add_edges(edges)

#         join_msg = anchor_cell.add_closest_edge_to_graph(base_ids, base_seg) 
        

#         # Update lists of base segments and displayed segs:
#         anchor_cell.cell_data['base_segments']['unknown'].update(set(base_ids))

#         with anchor_cell.viewer.txn(overwrite=True) as s:

#             for bs in base_ids:
#                 s.layers['base_segs'].segment_colors[int(bs)] = '#ff0000' #'#d2b48c'
#                 s.layers['base_segs'].segments.add(int(bs))
                
#         base_ids_added.update(base_ids)


#         anchor_cell.update_displayed_segs() 
#         anchor_cell.assert_segs_in_sync()

In [ ]:
base_seg

In [ ]:
anchor_cell.save_cell_graph()

## Check for duplicates in mg_network

In [ ]:
path_to_settings_json = '/Users/kperks/Documents/ell-connectome/eCREST-local-files/settings_dict.json'
settings_dict = import_settings(path_to_settings_json)

dirpath = Path(settings_dict['save_dir']) #/ 'todo_postsynaptic_mg/check-duplicates' #'todo_presynaptic'#
# dirpath = "/Users/kperks/Documents/gdrive/.shortcut-targets-by-id/16q1BuOMfD2ta0Cwq8CjMlRe4rDvbuWC5/ELL_connectome/CREST_reconstructions/mg-network"

nodes = [child.name.split('_')[2] for child in sorted(dirpath.iterdir()) 
         if (child.name[0]!='.') & (child.is_file())] # ignore hidden files]

nodefiles = dict()
for child in sorted(dirpath.iterdir()):
    if (child.name[0]!='.') & (child.is_file()):
        nodefiles[child.name.split('_')[2]] = child
                    

For each of the previous files,
Check against the following directory

In [ ]:

directory_list = [
    Path(settings_dict['save_dir']),
    # Path(settings_dict['save_dir'])/'volume-subsample-all/in-progress',
    # Path(settings_dict['save_dir'])/'todo_postsynaptic_grc',
    # Path(settings_dict['save_dir'])/'todo_postsynaptic_sg',
    # Path(settings_dict['save_dir'])/'todo_postsynaptic_mg',
    # Path(settings_dict['save_dir'])/'todo_presynaptic',
    # Path(settings_dict['save_dir'])/'todo_presynaptic/mg2_214581797',
    # Path(settings_dict['save_dir'])/'todo_presynaptic/mg1_299496636',
    # Path(settings_dict['save_dir'])/'todo_presynaptic/Krista/grc_386392158',
    # Path(settings_dict['save_dir'])/'todo_presynaptic/unsure',
    # Path(settings_dict['save_dir'])/'todo_presynaptic/needs-cell-type'
]

for d_ in directory_list:
    df_all = pd.DataFrame()
    crest = ecrest(settings_dict,launch_viewer=False)
    base_segments = crest.get_base_segments_dict(d_)# / 'todo_postsynaptic_sg/check-duplicates')


    for k,f in nodefiles.items():
        cell = ecrest(settings_dict,filepath = f,launch_viewer=False)
        df = cell.check_duplicates(base_segments)
        if not df.empty:
            df_all = pd.concat([df_all,df]) 
    
    print(f'for directory {d_} the following are duplicates with cells in main folder')
    display(df_all)
    print('')

In [ ]:
cellf = 'cell_graph_310752287__2023-07-26 19.49.58.json'
cell = ecrest(settings_dict,filepath = dirpath/cellf, launch_viewer=True) 

## visualize overlapping segments for duplicates

In [ ]:
path_to_settings_json = '/Users/kperks/Documents/ell-connectome/eCREST-local-files/settings_dict.json'
settings_dict = import_settings(path_to_settings_json)
dirpath = Path(settings_dict['save_dir'])
                  
# First, where is the "main" cell?
    # This will create a base_segments dictionary of all cells in this main directory
cell = ecrest(settings_dict,launch_viewer=False)
base_segments =  cell.get_base_segments_dict(dirpath)

base_segments_dup = base_segments# cell.get_base_segments_dict(dirpath / 'todo_postsynaptic_sg/check-duplicates')#'todo_postsynaptic_mg/check-duplicates') #base_segments #

In [ ]:
nodefiles = dict()
for child in sorted(dirpath.iterdir()):
    if (child.name[0]!='.') & (child.is_file()):
        nodefiles[child.name.split('_')[2]] = child

In [ ]:
main = '44025489'			
dup = '44025132' 		

overlap_segs={}
overlap_segs['main']=base_segments[main].difference(base_segments_dup[dup])
overlap_segs['dup']=base_segments_dup[dup].difference(base_segments[main])

print(f'{len(overlap_segs["main"])} segments in main on {nodefiles[main].name.split("_")[-1][:-5]} that are not in dup')
print(f'{len(overlap_segs["dup"])} segments in dup on {nodefiles[dup].name.split("_")[-1][:-5]} that are not in main')

overlap_seg_list = base_segments[main] & base_segments_dup[dup]#base_segments_dup[dup]
print(f'{len(overlap_seg_list)} segments in both')

# overlap_segs["dup"]
# overlap_segs["dup"] = overlap_segs["dup"].difference(set(['642149703']))

### Create viewer

In [ ]:
viewer = neuroglancer.Viewer()
viewer.set_state({})

location=[17000,17000,1500]

with viewer.config_state.txn() as s:
    s.show_layer_panel = True ###
with viewer.txn(overwrite=True) as s:
    dimensions = neuroglancer.CoordinateSpace(
        scales=[16, 16, 30],# self.vx_sizes['em'],
        units='nm',
        names=['x', 'y', 'z']   )
    s.showSlices = False
    s.dimensions = dimensions
    s.position = array(location)
    s.layout = "3d"
    s.projectionScale = 30000
    s.projection_background_color= "#000000"

with viewer.txn(overwrite=True) as s:
    wb_open(str(viewer))

db_cursors = sqlite3_connect(settings_dict['db_path'], check_same_thread=False).cursor()
a = ', '.join(['base_address'])
db_cursors.execute(f'''SELECT {a} FROM addresses_table LIMIT 1''')
[base_seg] = db_cursors.fetchall()[0]
two_d_intensity = 0.5

for layer_name in ['main','dup','overlap']:
    with viewer.txn(overwrite=True) as s:
        s.layers[layer_name] = neuroglancer.SegmentationLayer(source = base_seg, segments=[], segment_colors={})
        s.layers[layer_name].ignoreNullVisibleSet = False
        s.layers[layer_name].pick = False
        s.layers[layer_name].selectedAlpha = two_d_intensity #For 2D

### load cells and color overlap
cell_color={'main':'#33cc33','dup':'#cc33ff'}

for k in ['main','dup']:
    with viewer.txn(overwrite=True) as s:
        color_structure = cell_color[k] # blue
        for bs in overlap_segs[k]:
            s.layers[k].segments.add(int(bs))
            s.layers[k].segment_colors[int(bs)] = color_structure # blue

color_structure='#ff0000'
with viewer.txn(overwrite=True) as s:
    for bs in list(overlap_seg_list):
        s.layers['overlap'].segments.add(int(bs))
        s.layers['overlap'].segment_colors[int(bs)] = color_structure # blue

### Load main cell so can add segments from duplicate

In [ ]:
dirpath = Path(settings_dict['save_dir']) #/ 'todo_postsynaptic_sg/check-duplicates'
f_list = [f.name for f in dirpath.glob('*' + dup + '*')]
try: 
    len(f_list)==1
    main_cell = ecrest(settings_dict,filepath= dirpath / f_list[-1], launch_viewer=True)
except:
    print(f'more than one file for {main}')

In [ ]:
# main_cell.save_cell_graph()
main_cell.get_ctype("manual")

In [ ]:
# main_cell.cell_data['removed_base_segs']=set()
main_cell.max_num_base_added=1500

In [ ]:
'''
THIS VERSION OF HOW TO DO THIS IS CURRENTLY WORKING BEST
'''
anchor_cell = main_cell
base_ids_added = set()

for base_seg in overlap_segs["dup"].difference(overlap_segs["main"]): #overlap_segs["dup"].difference(overlap_segs["main"]): # dup diff main adds segments in dup that were not in main
    
    if (base_ids_added&set(base_seg)==set()) & (base_seg != anchor_cell.cell_data['metadata']['main_seg']['base']): 
        
        displayed_segs = anchor_cell.assert_segs_in_sync(return_segs=True)
        if base_seg in displayed_segs:
            # print(f'{base_seg} already in cell, continueing')
            continue

        # print(i,base_seg)
        agglo_seg = anchor_cell.get_agglo_seg_of_base_seg(base_seg)

        constituent_base_ids = anchor_cell.get_base_segs_of_agglo_seg(agglo_seg)        
        current_segs = anchor_cell.assert_segs_in_sync(return_segs=True)

        num_base_segs_this_agglo_seg = len(constituent_base_ids)
        constituent_base_ids = [x for x in constituent_base_ids if x not in current_segs]
        constituent_base_ids = [x for x in constituent_base_ids if x not in anchor_cell.cell_data['removed_base_segs']]
        num_base_segs_not_already_included = len(constituent_base_ids)
        
        if len(constituent_base_ids) > anchor_cell.max_num_base_added:
            base_ids = [base_seg]
            # anchor_cell.large_agglo_segs.add(agglo_seg)
            print(f'{len(constituent_base_ids)} other base segments in the agglo segment; max number can add is {anchor_cell.max_num_base_added}')
            # print(f'{base_seg} part of an agglo seg {agglo_seg} that is too large to add, so just adding the one segment')
        else:
            base_ids = constituent_base_ids

        if num_base_segs_this_agglo_seg > num_base_segs_not_already_included:

            if not base_seg in base_ids:
                base_ids.append(base_seg)
        print(base_ids)
        anchor_cell.update_base_locations(base_ids)
        anchor_cell.pr_graph.add_vertices(base_ids)

        if len(base_ids) > 1:
            edges = anchor_cell.get_edges_from_agglo_seg(agglo_seg)
            edges = [x for x in edges if (x[0] in base_ids and x[1] in base_ids)]
            anchor_cell.pr_graph.add_edges(edges)

        join_msg = anchor_cell.add_closest_edge_to_graph(base_ids, base_seg) 
        

        # Update lists of base segments and displayed segs:
        anchor_cell.cell_data['base_segments']['unknown'].update(set(base_ids))

        with anchor_cell.viewer.txn(overwrite=True) as s:

            for bs in base_ids:
                s.layers['base_segs'].segment_colors[int(bs)] = '#d2b48c'
                s.layers['base_segs'].segments.add(int(bs))
                
        base_ids_added.update(base_ids)


        anchor_cell.update_displayed_segs() 
        anchor_cell.assert_segs_in_sync()


In [ ]:
anchor_cell.define_ctype("sg2","manual")

In [ ]:
anchor_cell.save_cell_graph()

### get annotations from duplicate cell into main

In [ ]:
json_path = Path(settings_dict['save_dir']) #/ 'kp'#/'todo_presynaptic/mg1_299496636' #/ 'todo_postsynaptic_sg/47366615' #
filename = 'cell_graph_129851820__2023-08-09 13.43.19.json'#'cell_graph_306242528__2023-06-26 09.26.24.json'

crest_ann = ecrest(settings_dict,filepath= json_path / filename, launch_viewer=False)

In [ ]:
# for layer_name in anchor_cell.cell_data['end_points'].keys():
layer_names = ['natural end','exit volume','pre-synaptic','post-synaptic']
for l_ in layer_names:
    anchor_cell.cell_data['end_points'][l_] = crest_ann.cell_data['end_points'][l_]

    anchor_cell.load_annotation_layer_points()

In [ ]:
anchor_cell.save_cell_graph()

In [ ]:
anchor_cell.get_ctype("manual")

In [ ]:
anchor_cell.define_ctype("sg1","manual")

In [ ]:

all_base_segs = [str(a) for b in main_cell.cell_data['base_segments'].values() for a in b]

# self.update_base_locations(all_base_segs)

In [ ]:
len(all_base_segs)

In [ ]:
results = {}
batch_size=1000
base_segs = all_base_segs[0:10]

if len(base_segs) > 0:

    num_batches = int(len(base_segs)/batch_size)

    for batch in range(num_batches+1):

        q = ','.join([str(x) for x in base_segs[batch*batch_size:(batch+1)*batch_size]])

        # query = f"""SELECT seg_id, x, y, z FROM base_location WHERE seg_id IN ({q})"""
        QUERY = f"""
        SELECT
            cast(objects.id as INT64) as seg_id,
            sample_voxel.x as x,
            sample_voxel.y as y,
            sample_voxel.z as z,
        FROM
            `lcht-goog-connectomics.ell_roi450um_seg32fb16fb_220930.objinfo` as objects
        WHERE objects.id IN ({q})
        """
        main_cell.db_cursors.execute(query)

        this_batch = {str(x[0]): (int(x[1]), int(x[2]), int(x[3])) for x in main_cell.db_cursors.fetchall()}

        results.update(this_batch)


In [ ]:
results

## Combine annotations and/or base segments across different CREST files

Duplicate cell

In [ ]:
json_path = Path(settings_dict['save_dir']) #/ 'todo_pre-synaptic/sg2'
filename = 'cell_graph_305965235__2023-06-26 13.33.46.json'

crest_1 = ecrest(settings_dict,filepath= json_path / filename, launch_viewer=True)

In [ ]:
crest_1.save_cell_graph()

Main cell

In [ ]:
json_path = Path(settings_dict['save_dir'])
filename = 'cell_graph_386117124__2023-04-09 14.32.27.json'

crest_2 = ecrest(settings_dict,filepath= json_path / filename, launch_viewer=True)


In [ ]:
crest_2.save_cell_graph()

### Get missing segments from one into the other...
and adjust graph too (find missing edges and vertices... instead of making new graph, use old?)

In [ ]:
segs_1 = set([a for b in crest_1.cell_data['base_segments'].values() for a in b])
segs_2 = set([a for b in crest_2.cell_data['base_segments'].values() for a in b])

print(f'{len(segs_1.difference(segs_2))} segments in cell 1 that are not in cell 2')
print(f'{len(segs_2.difference(segs_1))} segments in cell 2 that are not in cell 1')

### add segments missing from one reconstruction to another

as loop... keeps track of all added (and exclude them from next iterations) because some can be in same agglo. 

In [ ]:
# assign which cell you want to add to (and then keep)
anchor_cell = crest_2

# assign which segments need to be added
base_ids_all = sorted(list(segs_1.difference(segs_2)))

In [ ]:
'''
THIS VERSION OF HOW TO DO THIS IS CURRENTLY WORKING BEST
'''

base_ids_added = set()

for base_seg in base_ids_all:
    
    if (base_ids_added&set(base_seg)==set()) & (base_seg != anchor_seg): 
        
        displayed_segs = anchor_cell.assert_segs_in_sync(return_segs=True)
        if base_seg in displayed_segs:
            # print(f'{base_seg} already in cell, continueing')
            continue

        # print(i,base_seg)
        agglo_seg = anchor_cell.get_agglo_seg_of_base_seg(base_seg)

        constituent_base_ids = anchor_cell.get_base_segs_of_agglo_seg(agglo_seg)        
        current_segs = anchor_cell.assert_segs_in_sync(return_segs=True)

        num_base_segs_this_agglo_seg = len(constituent_base_ids)
        constituent_base_ids = [x for x in constituent_base_ids if x not in current_segs]
        constituent_base_ids = [x for x in constituent_base_ids if x not in anchor_cell.cell_data['removed_base_segs']]
        num_base_segs_not_already_included = len(constituent_base_ids)
        
        if len(constituent_base_ids) > anchor_cell.max_num_base_added:
            base_ids = [base_seg]
            # anchor_cell.large_agglo_segs.add(agglo_seg)
            print(f'{len(constituent_base_ids)} other base segments in the agglo segment; max number can add is {anchor_cell.max_num_base_added}')
            # print(f'{base_seg} part of an agglo seg {agglo_seg} that is too large to add, so just adding the one segment')
        else:
            base_ids = constituent_base_ids

        if num_base_segs_this_agglo_seg > num_base_segs_not_already_included:

            if not base_seg in base_ids:
                base_ids.append(base_seg)
        
        anchor_cell.update_base_locations(base_ids)
        anchor_cell.pr_graph.add_vertices(base_ids)

        if len(base_ids) > 1:
            edges = anchor_cell.get_edges_from_agglo_seg(agglo_seg)
            edges = [x for x in edges if (x[0] in base_ids and x[1] in base_ids)]
            anchor_cell.pr_graph.add_edges(edges)

        join_msg = anchor_cell.add_closest_edge_to_graph(base_ids, base_seg) 
        

        # Update lists of base segments and displayed segs:
        anchor_cell.cell_data['base_segments']['unknown'].update(set(base_ids))

        with anchor_cell.viewer.txn(overwrite=True) as s:

            for bs in base_ids:
                s.layers['base_segs'].segment_colors[int(bs)] = '#d2b48c'
                s.layers['base_segs'].segments.add(int(bs))
                
        base_ids_added.update(base_ids)


        anchor_cell.update_displayed_segs() 
        anchor_cell.assert_segs_in_sync()


In [ ]:
crest_2.cell_data['end_points']['pre-synaptic'] = crest_1.cell_data['end_points']['pre-synaptic']
crest_2.load_annotation_layer_points()

### Create new crest file from the union segment list...

In [ ]:
new_seg_list = segs_1.union(segs_2)
segment_id = crest_1.cell_data['metadata']['main_seg']['base']

In [ ]:
combo_crest = ecrest(settings_dict, segment_id = segment_id, segment_list = new_seg_list, launch_viewer=True)

Add annotations from one of the cells...

In [ ]:
combo_crest.cell_data['end_points'] = crest_1.cell_data['end_points']

combo_crest.load_annotation_layer_points()

In [ ]:
combo_crest.define_ctype('uk','manual')

In [ ]:
combo_crest.save_cell_graph()

#### DONT FORGET TO SAVE YOUR WORK! 



## Other...

### Add vertex if missing (if can't remove a segment, sometimes this is the reason)

In [ ]:
# ('479295220')
crest.cell_data['base_segments']['unknown'].add('565168297')

In [ ]:
crest.pr_graph.vs.find("459940426")

In [ ]:
crest.pr_graph.add_vertex(name='459940426')

In [ ]:
crest.pr_graph.add_edges([(4966,323)])

### define cell type for a crest file

resaves as original file name (not with an updated timestamp)

In [ ]:
dirpath = Path('/Users/kperks/Documents/gdrive/.shortcut-targets-by-id/16q1BuOMfD2ta0Cwq8CjMlRe4rDvbuWC5/ELL_connectome/CREST_reconstructions/mg-network')
filepath = dirpath / 'cell_graph_307591597__2023-04-07 12.54.44.json'
cell_type = 'lf'

### 
crest = ecrest(settings_dict, filepath = filepath, launch_viewer=False);
crest.define_ctype(cell_type,'manual')
crest.get_ctype('manual') == cell_type
crest.save_cell_graph(directory_path = filepath.parent, file_name=filepath.name, save_to_cloud=False); #rewrites the original, not with a new time stamp

check cell type in neuroglancer

In [ ]:
dirpath = Path('/Users/kperks/Documents/gdrive/.shortcut-targets-by-id/16q1BuOMfD2ta0Cwq8CjMlRe4rDvbuWC5/ELL_connectome/CREST_reconstructions/mg-network')
filepath = dirpath / 'cell_graph_213605530__2023-03-29 22.49.21.json'

crest = ecrest(settings_dict, filepath = filepath, launch_viewer=True)

In [ ]:
crest.save_cell_graph(directory_path = filepath.parent, file_name=filepath.name, save_to_cloud=False); 

### get cell types of neuroglancer reconstructions into crest json files


In [ ]:
path_to_settings_json = '/Users/kperks/Documents/ell-connectome/eCREST-local-files/settings_dict.json'
settings_dict = import_settings(path_to_settings_json)

In [ ]:
crestpath = "/Volumes/GoogleDrive/.shortcut-targets-by-id/16q1BuOMfD2ta0Cwq8CjMlRe4rDvbuWC5/ELL_connectome/CREST_reconstructions/mg-network"
ngpath = "/Volumes/GoogleDrive/.shortcut-targets-by-id/16q1BuOMfD2ta0Cwq8CjMlRe4rDvbuWC5/ELL_connectome/CREST_reconstructions/mg-network/files_for_names"
ngfiles = [x.name for x in Path(ngpath).iterdir()]

In [ ]:
ctype_list = []
has_ctype = set()
all_cells = set()

for fname in sorted(list(Path(crestpath).iterdir())):
    if (fname.name[0]!='.') & (fname.is_file()):
        # display(fname.name)
        crest = ecrest(settings_dict, filepath = fname, launch_viewer=False);
        ngfile = list(filter(lambda x: cell.cell_data['metadata']['main_seg']['base'] in x, ngfiles))
        
        all_cells = all_cells | set({cell.cell_data['metadata']['main_seg']['base']})
        
        if len(ngfile)==1:
            ctype = ngfile[0].split('_')[3].lower()
            has_ctype = has_ctype | set({cell.cell_data['metadata']['main_seg']['base']})
        ctype_list.append(ctype)
        crest.define_ctype(ctype,'manual');
        crest.save_cell_graph(directory_path = fname.parent, file_name=fname.name, save_to_cloud=False);

In [ ]:
# make sure all crest cells have cell type definition from neuroglancer file name
all_cells-has_ctype

In [ ]:
# check cell type labels
list(unique(ctype_list))

### resave a json file with formatting for readability

In [ ]:
filepath = Path("D:\electric-fish\eCREST\CREST_settings.json")
with open(filepath, "r") as f:
    loaded = json.load(f)

with open(filepath, "w") as f:
    json.dump(loaded, f, indent=4)

### Add found missing segments to reconstructions

manually go through each cell with missing segments, search and add them...

keep a running "todo" list of any segments that should be a new reconstruction rather than missing from current

In [ ]:
missing_path = Path('/Users/kperks/Documents/gdrive/.shortcut-targets-by-id/16q1BuOMfD2ta0Cwq8CjMlRe4rDvbuWC5/ELL_connectome/CREST_reconstructions/mg-network/todo_post-synaptic/reconstructed_missing_segs-20230501.json')
with open(missing_path,'r') as fp:
    reconstructed_segs=fp.read()
    reconstructed_segs = json.loads(reconstructed_segs)


for each key in the dict, open the crest file for that cell (from nodefiles) and visualize the missing segments

In [ ]:
path_to_settings_json = '/Users/kperks/Documents/ell-connectome/eCREST-local-files/settings_dict.json'
settings_dict = import_settings(path_to_settings_json)

dirpath = Path(settings_dict['save_dir'])
# dirpath = "/Users/kperks/Documents/gdrive/.shortcut-targets-by-id/16q1BuOMfD2ta0Cwq8CjMlRe4rDvbuWC5/ELL_connectome/CREST_reconstructions/mg-network"

nodes = [child.name.split('_')[2] for child in sorted(dirpath.iterdir()) 
         if (child.name[0]!='.') & (child.is_file())] # ignore hidden files]

nodefiles = dict()
for child in sorted(dirpath.iterdir()):
    if (child.name[0]!='.') & (child.is_file()):
        nodefiles[child.name.split('_')[2]] = child
                    

In [ ]:
reconstructed_segs

In [ ]:
k = 36

crest = ecrest(settings_dict,filepath= nodefiles[keys[k]], launch_viewer=True)

print(keys[k], reconstructed_segs[keys[k]])

In [ ]:
with crest.viewer.config_state.txn() as s:
    s.input_event_bindings.data_view["alt+mousedown0"]="add-or-remove-seg"
    s.input_event_bindings.data_view["alt+mousedown2"]="mark-branch-in-colour"
    print(s.input_event_bindings.data_view)

In [ ]:
crest.save_cell_graph()

In [ ]:
crest.get_ctype('manual')

In [ ]:
todo = [564038367,116931244,128551991,129636736,558157595,130764619,474759791,49654133,390060758,49873267,563840123,130656616,135592261]